In [1]:
import qforte

# Example notebook for using QForte molecule class and using UCCSD

NOTE: Before running this notebook you will need to have installed Psi4, OpenFermion, and the OpenFermion-Psi4 plugin. There are all avalible via intall through conda-forge and/or pip.

Initialize a QForte molecule, and specify what backend to use. Currently there is only support for using Openfermion-Psi4, but will eventually include direct conections to Psi4 and Forte. First we need to define some input arguments. 

In [2]:
# System factory arguments
system_type = 'molecule'
build_type = 'openfermion'


# Molecule arguments
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.75))]
basis = 'sto-3g'
multiplicity = 1


# Run arguments for backend
run_scf = 1
run_mp2 = 1
run_ccsd = 1
run_fci = 1

We need to initialize an adapter object, wichin in trun is able to ruturn a molecule object.

In [3]:
# Initialize the adapter (specific to backend and system type)
adapter = qforte.system.system_factory(mol_geometry=geometry, basis=basis,
                                       multiplicity=1, description='h2_uccsd_test_075')

# Populate the attributes by calling run() with backend run args. we can store the ccsd
# amplitudes 
adapter.run(run_scf=run_scf, run_mp2=run_mp2, run_ccsd=run_ccsd,
                   run_fci=run_fci, store_uccsd_amps=True)


Using standard openfermion hamiltonian ordering!


Get the molecule from the adapter.

In [4]:
H2 = adapter.get_molecule()

From the backend, several values are avalible in the molecule for reference.

In [5]:
# Hamiltonian
Hamiltonian = H2.get_hamiltonian()
print('\nQubit Hamiltonian\n', Hamiltonian.str())

# Various energies from psi4 specified in run args
print('\nPsi4 Energies')
print('---------------------------------------------------')
print('hf_energy:                     ', H2.get_hf_energy())
print('mp2_energy:                    ', H2.get_mp2_energy())
print('ccsd_energy:                   ', H2.get_ccsd_energy())
print('fci_energy:                    ', H2.get_fci_energy())


Qubit Hamiltonian
 -0.109731[] +0.169885[Z0] +0.169885[Z1] -0.218863[Z2] -0.218863[Z3] +0.168212[Z0 Z1] +0.045443[Y0 X1 X2 Y3] -0.045443[X0 X1 Y2 Y3] -0.045443[Y0 Y1 X2 X3] +0.045443[X0 Y1 Y2 X3] +0.120051[Z0 Z2] +0.165494[Z0 Z3] +0.165494[Z1 Z2] +0.120051[Z1 Z3] +0.173954[Z2 Z3]

Psi4 Energies
---------------------------------------------------
hf_energy:                      -1.1161514489141386
mp2_energy:                     -1.129524145749418
ccsd_energy:                    -1.1371170673687787
fci_energy:                     -1.1371170673370452


Now we can actually run a method, such as uccsd optemized via VQE. Currently only supports gradient-free optemization such as Nelder-Mead.

In [6]:
# First we need a reference state (Hartee-Fock usually). 
# In QForte, qubits are aranged as [alpha1, beta1, alpha2, beta2, .... ]
ref = [1,1,0,0]

usscd_energy = qforte.ucc.uccsd.uccsd_ph_energy_opt(H2, ref, maxiter = 5000)

AttributeError: module 'qforte.ucc' has no attribute 'uccsd'